In [1]:
import torch
import os
import pandas as pd
import matchzoo as mz
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
print(mz.__version__)
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import nltk
nltk.download('punkt')
# nltk.set_proxy('SYSTEM PROXY')

1.1.1


[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>


False

In [3]:
TYPE = 'classification'

classification_task = mz.tasks.Classification(num_classes=3)
classification_task.metrics = ['acc']
print(classification_task.num_classes)
print(classification_task.output_shape)
print(classification_task.output_dtype)
print(classification_task)

3
(3,)
<class 'int'>
Classification Task with 3 classes


# Data Pack Sample

In [4]:
# pack
df = pd.DataFrame(data={'text_left': list('ABBCD'),
                       'text_right': list('aacbd'),
                       'label': [-1, 1, 0, 0, 1]})
mz.pack(df, task=TYPE).frame()

,id_left,text_left,id_right,text_right,label
0,L-0,A,R-0,a,-1
1,L-1,B,R-0,a,1
2,L-1,B,R-1,c,0
3,L-2,C,R-2,b,0
4,L-3,D,R-3,d,1


In [5]:
# data_pack
left = [
    ['artid1', 'A1'],
    ['artid2', 'A2'],
    ['artid3', 'A3']
]
right = [
    ['hypoid1', 'prompt1'],
    ['hypoid2', 'prompt2'],
    ['hypoid3', 'prompt3']
]
relation = [
    ['artid1', 'hypoid1', -1],
    ['artid1', 'hypoid3', 1],
    ['artid2', 'hypoid2', 0],
    ['artid3', 'hypoid3', 1]
]

relation_df = pd.DataFrame(relation)
# relation_df
left = pd.DataFrame(left)
right = pd.DataFrame(right)
dp = mz.DataPack(
    relation=relation_df,
    left=left,
    right=right
)
# print(len(dp))
# print(type(dp.frame))
# frame_slice = dp.frame[0:5]
# type(frame_slice)
# list(frame_slice.columns)
# full_frame = dp.frame()

In [6]:
data_pack = mz.datasets.toy.load_data(stage='train')
type(data_pack)

matchzoo.data_pack.data_pack.DataPack

# Prepare input data

In [7]:
"""Annotation file"""
annot = pd.read_csv('../annotations/annotations_merged.csv')
print(annot.dtypes)
annot.sort_values('PMCID').head(10)

UserID              int64
PromptID            int64
PMCID               int64
Valid Label          bool
Valid Reasoning      bool
Label              object
Annotations        object
Label Code          int64
In Abstract          bool
Evidence Start      int64
Evidence End        int64
dtype: object


,UserID,PromptID,PMCID,Valid Label,Valid Reasoning,Label,Annotations,Label Code,In Abstract,Evidence Start,Evidence End
13994,0,7902,60007,True,True,significantly increased,Mivacurium 250 μg/kg produced a maximal T bloc...,1,True,1280,1395
14000,3,7904,60007,True,True,no significant difference,Heart rate was similar between doses,0,True,1452,1488
12483,0,7905,60007,True,True,significantly decreased,while both AUC-SBP and AUC-DBP were significan...,-1,False,16737,16836
12484,1,7905,60007,True,True,significantly decreased,"In relation to the cardiovascular response, th...",-1,False,16641,16836
13999,0,7904,60007,True,True,no significant difference,"In relation to the cardiovascular response, th...",0,False,16641,16736
13998,3,7903,60007,True,True,no significant difference,Spontaneous recovery times were similar in bot...,0,True,1396,1450
13997,0,7903,60007,True,True,no significant difference,The times to OA and to spontaneous recovery of...,0,False,14755,15079
13995,3,7902,60007,True,True,significantly increased,Mivacurium 250 μg/kg produced a maximal T bloc...,1,True,1280,1395
2855,0,1806,111193,True,True,no significant difference,The amount of blood transfusion was identical ...,0,True,1418,1578
2334,0,1808,111193,True,True,no significant difference,There was no difference in time spent in hospi...,0,False,14701,14770


In [8]:
"""Prompt file"""
prompt = pd.read_csv('../annotations/prompts_merged.csv')
print(prompt.dtypes)
prompt.sort_values('PMCID').head(10)

PromptID         int64
PMCID            int64
Outcome         object
Intervention    object
Comparator      object
dtype: object


,PromptID,PMCID,Outcome,Intervention,Comparator
7217,7902,60007,maximal T block,mivacurium 250 μg/kg,mivacurium 200 μg/kg
6459,7905,60007,systolic blood pressure (SBP) and diastolic bl...,mivacurium 250 μg/kg,mivacurium 200 μg/kg
7219,7903,60007,Spontaneous recovery time,mivacurium 250 μg/kg,mivacurium 200 μg/kg
7220,7904,60007,heart rate (HR),mivacurium 250 μg/kg,mivacurium 200 μg/kg
1458,1804,111193,"Blood loss, excluding intraoperative blood loss",Preoperative acute normovolaemic hemodilution ...,No preoperative hemodilution (non-ANH)
1459,1806,111193,Amount of transfused blood,Preoperative acute normovolaemic hemodilution ...,No preoperative hemodilution (non-ANH)
1308,1807,111193,Mean artery blood pressure during surgery,Preoperative acute normovolaemic hemodilution ...,No preoperative hemodilution (non-ANH)
1193,1808,111193,Hospital stay,Preoperative acute normovolaemic hemodilution ...,No preoperative hemodilution (non-ANH)
1192,1805,111193,Surgery duration,Preoperative acute normovolaemic hemodilution ...,No preoperative hemodilution (non-ANH)
467,550,125315,The 28-day mortality,received standard therapy for sepsis and physi...,received only standard therapy for sepsis


In [9]:
"""Concatenate prompt for input"""
prompt['Prompt'] = [str(triplet)[1:-1] for triplet in zip(prompt['Outcome'].values,
                                             prompt['Intervention'].values,
                                             prompt['Comparator'].values)]
# print(type(prompt['Prompt'][0]), type(prompt['Outcome'][0]))
# print(prompt['Intervention'][0])
prompt.head()

,PromptID,PMCID,Outcome,Intervention,Comparator,Prompt
0,213,2206488,IL-6r (ng/ml),dextran-70 infusion was administered at a dose...,identical amounts of gelatin infusion,"'IL-6r (ng/ml)', 'dextran-70 infusion was admi..."
1,98,2858204,downsizing of ucler area 2 weeks after treatment,HBOT,placebo,'downsizing of ucler area 2 weeks after treatm...
2,150,2871176,Incidence of minor hypoglycaemia,Liraglutide (1.2 mg) plus glimepiride,Rosiglitazone plus glimepiride,"'Incidence of minor hypoglycaemia', 'Liragluti..."
3,191,2517154,Dysmenorrhea scores,Infliximab,Placebo,"'Dysmenorrhea scores', 'Infliximab', 'Placebo'"
4,113,2871176,Patients reaching HbA1c goals less than 7.0% a...,Liraglutide (1.8 mg) plus glimepiride,Rosiglitazone plus glimepiride,'Patients reaching HbA1c goals less than 7.0% ...


In [10]:
"""Process txt file for Articles input"""
TXT_PATH = '../annotations/txt_files/'
TAR_PATH = '../annotations/processed_txt_files/'
look_up = {}

if not os.path.exists(TAR_PATH):
    os.mkdir(TAR_PATH)
# else: os.removedirs(TAR_PATH)
for file in os.listdir(TXT_PATH):
    fname = file[3:]
    look_up[int(file[3:-4])] = fname
    with open(TXT_PATH+file, 'r', encoding='utf-8') as f, open(TAR_PATH+fname, 'w', encoding='utf-8') as t:
        for line in f.readlines():
            t.write(line.strip())

In [11]:
"""Articles Map"""
left_articles = []
for file in os.listdir(TAR_PATH):
    with open(TAR_PATH+file, 'r', encoding='utf-8') as f:
        left_articles.append([int(file[:-4]), f.readlines()[0]])
# print(left_articles[0:5])

"""Prompts Map"""
right_prompts = [list(pair) for pair in zip(annot['PromptID'].values, annot['Annotations'].values)]
print(right_prompts[0:5])

"""Articles <-> Prompts Map"""
article_prompt_relations = [list(triplet) for triplet in zip(annot['PMCID'].values, 
                                                             annot['PromptID'].values, 
                                                             annot['Label Code'].values)]
print(article_prompt_relations[0:2])

"""Create Data-pack"""
left = pd.DataFrame(left_articles, columns=['id_left', 'text_left'])
right = pd.DataFrame(right_prompts, columns=['id_right', 'text_right'])
relation = pd.DataFrame(article_prompt_relations, columns=['id_left', 'id_right', 'label'])
dp = mz.DataPack(
    relation = relation,
    left = left,
    right = right
)

# print(left)
# print(len(dp))
# print(type(dp.frame))
# frame = dp.frame
# print(list(frame().columns))
# # frame_slice = dp.frame[0:5]
# # type(frame_slice)
# # list(frame_slice.columns)
# full_frame = dp.frame()

[[213, 'IL-6r (ng/ml)\t\t\t\t\t\t\t Group A\t43.6 (1.7–125.0)\t\t47.4 (0.7–109.5)\t56.2 (25.2–226.3)\t\t\t0.949† Group B\t40.7 (15.6–94.6)\t\t42.4 (22.2–100.5)\t50.2 (13.2–104.9)\t\t\t0.861†\tp = 0.607*\t\tp = 0.914*\tp = 0.304*'], [213, 'There was no significant difference in IL 6, IL-6r and C-reactive protein values between groups.'], [213, 'There was no significant difference in IL 6, IL-6r and C-reactive protein values between groups'], [213, 'There was no significant difference in IL 6, IL-6r and C-reactive protein values between groups'], [98, 'After two weeks of treatment, the reduction in ulcer area was doubled in the HBOT group (P = 0.037)']]
[[2206488, 213, 0], [2206488, 213, 0]]


In [12]:
""" Read article by name """
def read_article(id) -> str:
    with open(TAR_PATH+look_up[id], 'r', encoding='utf-8') as f:
        return f.readlines()[0]

text_left = [read_article(id) for id in annot['PMCID']]
# print(text_left[0:5])

In [13]:
df = pd.DataFrame(data={
    'id_left': annot['PMCID'].astype(str),
    'text_left': text_left,
    'id_right': annot['PromptID'].astype(str),
    'text_right': prompt['Prompt'],
    'label': annot['Label Code']+1
})
print(df.dtypes)

""" Split data pack into train/valid """
train, valid = train_test_split(df, test_size=0.2)
train_pack = mz.pack(train, task=TYPE)
valid_pack = mz.pack(valid, task=TYPE)
train_pack.frame().head(10) # DataFrame

id_left       object
text_left     object
id_right      object
text_right    object
label          int64
dtype: object


,id_left,text_left,id_right,text_right,label
0,4251531,TITLE: BP-C1 in the treatment of patients with...,3673,"'Alkaline phosphatase', 'Dimethylamylamine sup...",2
1,4292865,TITLE: A Prospective Randomized Controlled Stu...,4178,"'Pain intensity at rest', 'Kinesio taping ', '...",1
2,4598910,TITLE: The effect of home-based inspiratory mu...,6177,"'Total blood loss', 'Tranexamic acid', 'Control'",1
3,3395326,TITLE: The Use of Epoetin-ABSTRACT:Introductio...,3758,"'EQ-5D Self-Care improvement ', 'Naloxegol ', ...",1
4,3921228,TITLE: Real-Time fMRI Neurofeedback Training o...,2074,"'Voiding symptoms', 'Hydrodistension plus intr...",0
5,3536709,TITLE: Low-protein vegetarian diet does not ha...,3168,"'Insulin half life', 'Antioxidant supplementat...",2
6,4895420,TITLE: Gallium Maltolate as an Alternative to ...,5520,'concentrations of the final catabolic product...,1
7,3407981,TITLE: Recovery from Cycling Exercise: Effects...,4998,"'Rates of HBsAg clearance', 'peginterferon alf...",2
8,5018700,TITLE: Effects of different dosages of oxycodo...,4389,'Levels of neutrophil gelatinase-associated li...,1
9,5288524,TITLE: Benefit from the Chin-Down Maneuver in ...,4426,"'Muscle creatine phosphate', 'Creatine supplem...",2


In [14]:
dp = mz.pack(df, task=TYPE)
print(type(dp.frame))
frame_slice = dp.frame[0:5]
print(list(frame_slice.columns))
full_frame = dp.frame()
len(full_frame) == len(dp)

<class 'matchzoo.data_pack.data_pack.DataPack.FrameView'>
['id_left', 'text_left', 'id_right', 'text_right', 'label']


True

# Model and Train 

In [15]:
preprocessor = mz.models.DIIN.get_default_preprocessor()
train_processed = preprocessor.fit_transform(train_pack)
valid_processed = preprocessor.transform(valid_pack)

print(preprocessor.context)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1314/1314 [01:19<00:00, 16.56it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 4881/4881 [00:02<00:00, 1694.36it/s]
Building Vocabulary from a datapack.: 100%|██████████| 34579136/34579136 [00:15<00:00, 2193649.39it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1314/1314 [01:07<00:00, 19.55it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 4881/4881 [00:01<00:00, 3631.62it/s]
Processing length_right with len: 100%|██████████| 4881/4881 [00:00<00:00, 434870.49it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 981/981 [00:41<00:00, 23.86it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1833/1833 [00:01<00:00, 172

{'ngram_process_unit': <matchzoo.preprocessors.units.ngram_letter.NgramLetter object at 0x1a30ec4b00>, 'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter object at 0x1a30ec49b0>, 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary object at 0x1a30ec4780>, 'vocab_size': 71552, 'embedding_input_dim': 71552, 'ngram_vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary object at 0x1a89b9d978>, 'ngram_vocab_size': 101}


In [16]:
ngram_callback = mz.dataloader.callbacks.Ngram(preprocessor, mode='index')

trainset = mz.dataloader.Dataset(
    data_pack=train_processed,
    mode='pair',
    num_dup=1,
    num_neg=4,
    callbacks=[ngram_callback]
)
validset = mz.dataloader.Dataset(
    data_pack=valid_processed,
    mode='point',
    callbacks=[ngram_callback]
)

In [17]:
padding_callback = mz.models.DIIN.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
validloader = mz.dataloader.DataLoader(
    dataset=validset,
    stage='dev',
    callback=padding_callback
)

In [18]:
model = mz.models.DIIN()
model.params['task'] = classification_task
model.params['embedding_output_dim'] = 100
model.params['embedding_input_dim'] = preprocessor.context['embedding_input_dim']
model.params['dropout_rate'] = 0.2
model.guess_and_fill_missing_params()
model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DIIN(
  (embedding): Embedding(71552, 100, padding_idx=0)
  (char_embedding): CharacterEmbedding(
    (char_embedding): Embedding(100, 8)
    (conv): Conv1d(8, 100, kernel_size=(5,), stride=(1,))
  )
  (exact_maching): Matching()
  (left_encoder): SemanticComposite(
    (att_linear): Linear(in_features=603, out_features=1, bias=False)
    (z_gate): Linear(in_features=402, out_features=201, bias=True)
    (r_gate): Linear(in_features=402, out_features=201, bias=True)
    (f_gate): Linear(in_features=402, out_features=201, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (right_encoder): SemanticComposite(
    (att_linear): Linear(in_features=603, out_features=1, bias=False)
    (z_gate): Linear(in_features=402, out_features=201, bias=True)
    (r_gate): Linear(in_features=402, out_features=201, bias=True)
    (f_gate): Linear(in_features=402, out_features=201, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (matching): Matching()
  (conv): Conv2d(201, 60, kern

In [19]:
optimizer = torch.optim.Adam(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=validloader,
    epochs=10
)

trainer.run()

KeyboardInterrupt: 